In [ ]:
import numpy as np
import pandas as pd


import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jan-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jan-2021/test.csv')
sub=pd.read_csv("../input/tabular-playground-series-jan-2021/sample_submission.csv")

In [ ]:
for i in train.columns[1:]:
    plt.figure(figsize=(10,5))
    sns.distplot(train[i])
    plt.show()

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(train.corr())

In [ ]:
from sklearn.metrics import mean_squared_error as mse
from  sklearn.tree import DecisionTreeClassifier
from  sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

In [ ]:
def rmse(predictions,target):
    return np.sqrt(mse(predictions,target))

In [ ]:
Xt,Xv,Yt,Yv = train_test_split(train.iloc[:,1:-1],train.iloc[:,-1],test_size=0.2,shuffle=False)

xgb = xgb.XGBRegressor()
 
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]
              }


xgb_grid = GridSearchCV(xgb,
                        parameters,
                        scoring = 'mse',
                        cv = 5, #specify the cv strategy, default is 5 sets
                        n_jobs = -1,  #using all the processor in parallel
                        verbose=True)


xgb_grid.fit(train.iloc[:,1:-1],
         train.iloc[:,-1])

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

In [ ]:
Xt,Xv,Yt,Yv = train_test_split(train.iloc[:,1:-1],train.iloc[:,-1],test_size=0.2,shuffle=False)


model = xgb.XGBRegressor(n_estimators=10000,learning_rate=0.05)
eval_set = [(Xv,Yv)]
model.fit(Xt,Yt, early_stopping_rounds=10,eval_metric= 'rmse', eval_set = eval_set, verbose = 100)

In [ ]:
pred = model.predict(test.iloc[:,1:])

In [ ]:
sub.target = pred

In [ ]:
sub.to_csv('submission.csv',index=False)